# Mixture model experiment
In this notebook we perform the experiment to verify that we indeed see an exponential amount of gradient queries for stochastic gradient descent with an increasing parameter $d$, whereas we observe a linear relation in the case of the Metropolis-adjusted Langevin algorithm.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import math
import numpy as np
import pandas as pd
import seaborn as sns
from src.mixture import GaussianMixture, random_ball, DirichletMixture
from src.optimizer import em, ula
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.lines import Line2D

## Generating the data
We begin by generating the datasets used in our evaluation. Since we are trying various distributions, we have to make sure that they are all parameterized properly so as to achieve the following properties:
1. The clusters need to be adequately separated.
2. The amount of clusters in our dataset needs to be few ($M=\log_2 d$ in paper).

Something we could consider trying out would be to violate these properties to see what the failure mode is.

In our experiment we use the following distributions (we always let $N=2^d$):
1. Gaussian $\sigma = 1 / \sqrt{d}, M=\log_2 d$
2. Dirichlet
3. Exponential
4. Student's T (included since not log-concave for all parameters)

For our experiment to work, we need to generate new such problems for an increasing parameter $d$. Therefore we will need in total $4d$ datasets to work with.

## Experiment
Now we begin the experiment. The setup is quite simplistic:
1. Iterate over parameter $d$.
2. Gather our distribution datasets for that parameter $d$.
3. Estimate the parameters of the distribution using expectation-maximization.
4. Estimate the parameters of the distribution using MALA.
5. Save the amount of gradient queries required for both approaches.
6. Create line plot of required gradient queries for convergence.

In [ ]:
def plot_points_in_ball(num_points: int = 1000, radius: int = 1, dim: int = 2):
    if dim < 2 or dim > 3:
        raise ValueError('Invalid dimension')

    points = random_ball(num_points, dim, radius=radius)

    subplot_kw = {}

    if dim == 3:
        subplot_kw=dict(projection='3d')

    fig, ax = plt.subplots(subplot_kw=subplot_kw)

    if dim == 2:
        ax.set_aspect('equal')
        patch = Circle((0, 0), radius, fill=False, ls='-', lw=0.25)
        ax.add_patch(patch)
        ax.axvline(c='grey', lw=0.5)
        ax.axhline(c='grey', lw=0.5)

    ax.scatter(*np.split(points, dim, axis=1), marker='.')
    plt.show()

In [ ]:
plot_points_in_ball(radius=2, dim=2)

In [ ]:
plot_points_in_ball(radius=2, dim=3)

In [ ]:
def plot_scatter(ax, points, marker = 'o', c = None, s = None):
    ax.scatter(*np.split(points, 2, axis=1), edgecolors=None, c=c, s=s, marker=marker)
    ax.axvline(c='grey', lw=0.5)
    ax.axhline(c='grey', lw=0.5)

def plot_axis(ax, model):
    ax.set_aspect('equal')
    patch = Circle((0, 0), radius=model.R, fill=False, ls='-', lw=0.25)
    ax.add_patch(patch)
    plot_scatter(ax, model.points, c='b', s=100)
    plot_scatter(ax, model.params, c='r', marker='o', s=30)
                
def plot_gmm_initializations_2d_2axes():
    gmm_init_from_data = GaussianMixture(2)
    gmm_init_random_ball = GaussianMixture(2, init_from_data=False)
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    
    ax1.set_title('Initialization from data points')
    ax2.set_title('Random initialization within ball')
    
    plot_axis(ax1, gmm_init_from_data)
    plot_axis(ax2, gmm_init_random_ball)
    
    # Custom legend
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Points', markerfacecolor='b', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Mean parameter', markerfacecolor='r', markersize=10)
    ]
    fig.legend(handles=legend_elements)

In [ ]:
plot_gmm_initializations_2d_2axes()

In [ ]:
def plot_scatter(ax, points, marker = 'o', c = None, s = None):
    ax.scatter(*np.split(points, 2, axis=1), c=c, s=s, marker=marker)

def plot_gmm_initializations_2d():
    gmm = GaussianMixture(2)
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    
    # plot x and y axes
    ax.axvline(c='grey', lw=0.5)
    ax.axhline(c='grey', lw=0.5)
    
    # plot ball of radius R
    patch = Circle((0, 0), radius=gmm.R, fill=False, ls='-', lw=0.25)
    ax.add_patch(patch)
    
    # plot points
    plot_scatter(ax, gmm.points, c='b', s=100)
    
    # plot two types of mean initializations
    plot_scatter(ax, gmm.init_params(from_data=False), c='g', s=30)
    plot_scatter(ax, gmm.init_params(from_data=True), c='r', s=30)
    
    ax.set_title('Mean parameter initialization schemes for GMM (d=2)')
    
    # Custom legend
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Points', markerfacecolor='b', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Random initialization', markerfacecolor='g', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Initialization from data', markerfacecolor='r', markersize=10)
    ]
    ax.legend(handles=legend_elements, loc='best')

In [ ]:
plot_gmm_initializations_2d()

In [ ]:
MAX_ITERATIONS = 5000
MAX_ESTIMATION_ITERATIONS = 1000000
EM_ESTIMATE_THRESHOLD = 1e-8
EM_ERROR_THRESHOLD = 1e-6
ULA_PARAM_ESTIMATE_THRESHOLD = 1e-5
ULA_OBJECTIVE_ESTIMATE_THRESHOLD = 1e-8
FROM_DATA = True
NB_TRIALS = 10
NB_TRIALS_ESTIMATE = 20
NB_EXPS_ULA = 1
MAX_D = 8
ERROR_GRADIENT_ESTIMATE = 1e-4
GAMMA = None

dims = range(2, MAX_D+1)

models = {
    m.__name__: {
        'constructor': m,
        'df': None
    }
    for m in [
        GaussianMixture
    ]
}

for name, model_dict in models.items():
    # Record results
    model_results = []

    
    # Default iterations to start with
    iterations_em = 2
    iterations_ula = 2
    
    # Run for rest of dimensions
    for d in tqdm(dims):
        # Create finite mixture model problem
        model = model_dict['constructor'](d)
        
        # Run EM
        if iterations_em < MAX_ITERATIONS:
            
            # Estimate underlying set of parameters
            em_true_params = None
            em_true_params_found = False
            em_true_params_iterations = iterations_em*1000
            while not em_true_params_found:
                
                # Assume we have enough iterations until proven otherwise
                em_true_params_found = True
                
                # Run through trials
                em_estimate_params = np.zeros((NB_TRIALS_ESTIMATE, em_true_params_iterations, 
                                               model.params.shape[0], model.params.shape[1]))
                em_estimate_objective = np.zeros((NB_TRIALS_ESTIMATE, em_true_params_iterations))
                for i in range(NB_TRIALS_ESTIMATE):
                    
                    # Exit if we failed for given amount of iterations already
                    if not em_true_params_found:
                        break
                    
                    # Reset parameters
                    model.reset()
                    
                    # Run EM
                    em_param_iterates, em_objective_iterates = em(model, em_true_params_iterations)
                    em_estimate_params[i] = em_param_iterates
                    em_estimate_objective[i] = em_objective_iterates
                    
                    # Make sure we don't differ too much from previous estimates
                    for j in range(i+1):
                        
                        # If differ by more than threshold, we reset and increase amount of iterations
                        delta = np.linalg.norm(em_estimate_params[j][-1] - em_estimate_params[i][-1], 1)
                        if delta > EM_ESTIMATE_THRESHOLD:
                            em_true_params_iterations *= 10
                            em_true_params_found = False
                            print(f'Delta: {delta}')
                            print(f'Iterations now: {em_true_params_iterations}')
                            break
                        
                # Just choose any of the 20 iterations
                em_true_params = em_estimate_params[-1]
                em_true_params_objective = em_estimate_objective[-1]
                
            # Repeat for the specified amount of trials
            for _ in range(NB_TRIALS):
            
                # Record how many iterations were required
                model.reset()
                iterations_em = len(em(model, MAX_ITERATIONS, em_true_params_objective[-1], EM_ERROR_THRESHOLD)[0])
                model_results.append({'Dimensions': d, 'Algorithm': 'EM', 'Gradient queries': iterations_em})
                
        # Run ULA (disabled as we couldn't get it to converge in the end)
        if False and iterations_ula < MAX_ITERATIONS:
            
            # Estimate underlying set of parameters
            ula_expected_params = None
            ula_expected_objective = None
            ula_expected_params_found = False
            ula_expected_params_iterations = iterations_ula*1000
            if ula_expected_params_iterations > MAX_ESTIMATION_ITERATIONS:
                raise TimeoutError
            while not ula_expected_params_found:
                
                # Assume we have enough iterations until proven otherwise
                ula_expected_params_found = True
                
                # Run through trials
                ula_estimate_params = np.zeros((NB_TRIALS_ESTIMATE,
                                               model.params.shape[0], model.params.shape[1]))
                ula_estimate_objective = np.zeros(NB_TRIALS_ESTIMATE)
                for i in range(NB_TRIALS_ESTIMATE):
                    
                    # Reset parameters
                    model.reset(FROM_DATA)
                    
                    # Run ULA
                    ula_param_iterates, ula_objective_iterates = ula(model, 
                                                                     ula_expected_params_iterations,
                                                                     NB_EXPS_ULA,
                                                                     ERROR_GRADIENT_ESTIMATE,
                                                                     gamma=GAMMA)
                    
                    # Compute expectation on parameters
                    ula_estimate_params[i] = np.mean(ula_param_iterates)
                    ula_estimate_objective[i] = np.mean(ula_objective_iterates)
                    
                    # Make sure we don't differ too much from previous estimates
                    for j in range(i+1):
                        
                        # If differ by more than threshold, we reset and increase amount of iterations
                        delta_params = np.linalg.norm(ula_estimate_params[j] - ula_estimate_params[i], 1)
                        delta_objective = abs(ula_estimate_objective[j] - ula_estimate_objective[i])
                        if delta_params > ULA_PARAM_ESTIMATE_THRESHOLD or delta_objective > ULA_OBJECTIVE_ESTIMATE_THRESHOLD:
                            ula_expected_params_iterations *= 10
                            print(f'Delta params: {delta_params}')
                            print(f'Delta objective: {delta_objective}')
                            print(f'Iterations now: {ula_expected_params_iterations}')
                            ula_expected_params_found = False
                            break
                        
                # Just choose any of the 20 iterations
                ula_expected_params = ula_estimate_params[-1]
                ula_expected_objective = ula_estimate_objective[-1]
                
            # Repeat for the specified amount of trials
            for _ in range(NB_TRIALS):
            
                # Record how many iterations were required
                model.reset(FROM_DATA)
                iterations_ula = len(ula(model, MAX_ITERATIONS, NB_EXPS_ULA, ERROR_GRADIENT_ESTIMATE, 
                                        exp_mu=ula_expected_params, exp_U=ula_expected_objective, 
                                         timeout=MAX_ITERATIONS, gamma=GAMMA)[0])
                model_results.append({'Dimensions': d, 'Algorithm': 'ULA', 'Gradient queries': iterations_ula})
                
                
    # Save results
    model_dict['df'] = pd.DataFrame(model_results)

In [ ]:
model_dict['df']

## Analysis
Now that we have all the necessary data we can examine our results to see if they make sense in the context of the paper we are referencing.

We begin by generating figures for the individual mixture problems:

In [ ]:
# Gaussian
plt.figure()
sns.lineplot(data=models['GaussianMixture']['df'], x='Dimensions', y='Gradient queries', hue='Algorithm').set_title('Gaussian Mixture Model')
plt.show()